# Estimated Time of Delay ✈ 
## ML Based Flight Departure Delay Prediction 
### Group Project (4 members)

## 1. Introduction

### 1.1 Project Abstract

According to the Bureau of Transportation Statistics, in August 2022, approximately 25% of flights to US airports arrived 15 minutes or more late<sup>1</sup>. These delays generate financial losses and impact customer experience. The ability to predict flight delays will allow the operations teams to take earlier action to accommodate customers, which is the goal of this project.

The source data used is the instructor-provided post-join data set with ~42 million rows. A rolling time series split cross validation was used during training. Linear regression, Decision tree, Random Forest and Gradient-Boosted Tree regression models were developed to predict departure delays. Gradient-Boosted Tree produced the best result in terms of prediction accuracy, but the Decision Tree model produces performance close to it but at a significantly more reasonable time. A performance/training time tradeoff will be discussed. A classfication task was also performed whether or not the flight would experienced a delay. Multiperceptron neural network with one hidden layer gave the best result.

### 1.2 Business Case

The prediction task can be viewed as a regression problem where the goal is to predict the actual delay time of a flight in minutes, or a classification task in which the model predicts whether there will be delay or not. A flight is regarded as being delayed if it is delayed 15mins or greater. The ability of the airline operations teams to predict if a flight will be delayed and how long the delay will be provides opportunities to take earlier actions to accomodate customer needs. By proactively offering alternative flight recommendations, discounts, or lounge vouchers after a flight delay prediction, customer experience will be improved, which can maintain customer demand and mitigate financial losses for the airline.

### 1.3 Research Question / Hypothesis

This machine learning project experimented on various prediction models to solve the delayed flights business case scenario. The main question we will be answering is:

*What customized features of machine learning pipelines best accurately predict delayed flights?*

Our hypothesis is that adding engineered features to our pipelines will increase a model's performance. Undersampling, oversampling and SMOTE strategies to correct an imbalanced binary class distribution will also affect model performances. Oversampling increases likelihood of a model to overfit and may impact performance. We will experiment on various regression models such as Linear Regression, Decision Tree, Random Forest, Gradient Boosted Tree, as well as classification models such as Decision Tree, Random Forest, Gradient Boosted Tree and Multilayer Perceptron Neuron Network. Various network architecture will be explored.  Experimental findings will be discussed in later sections.

## 2. Data and Feature Engineering


### 2.1 Data Summary

We chose to use the instructor-provided data set, which already has joined the individual stations, and flights data with the weather data which has 42,271,633 rows and 168 columns. A summary of the count of flights in each class of delay length is shown below and shows that the vast majority of flights are on time or early, and those that are delayed are generally not delayed by more than 15-30 minutes. 

<img src="https://github.com/stackoverthrow/261section7group2/blob/main/class_distribution.jpeg?raw=true" width=80%>

### 2.2 Data Lineage

The working data is comprised from three separature individual data frames below. During the initial phases of the project we produced our own joined data set following the strategy shown in the diagram below. To enable more consistent results with our peers in the leader boards and to enable more focus on the model tuning we decided to use the instructor joined data set. Based on the names of the columns in the data set and number of rows it seems the join strategy is very similar to the one we employed in our phase 3 notebook. After loading the instructor data set we separately engineered several sub-dataframes representing the engineered features. For example the volume_sigma is calculated on an airport and day grouping and then the resultant volume_sigma value is joined to the main data set for all flights with the same date and airport. There were a few missing results in the engineered features resulting in loss of 27 rows when joining back to the main data set. This loss of data is insignificant relative to the 42M+ rows that remain. 
* Flights data from [BTS](https://www.transtats.bts.gov/DatabaseInfo.asp?QO_VQ=EFD&DB_URL=)
* Weather data from [NCEI](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00679)
* Weather station data [DOT](dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data)

<img src="https://github.com/stackoverthrow/261section7group2/blob/main/data_lineage_table.jpeg?raw=true" width=70%>

### 2.3 List of Family Features

The list of family features and their EDA are summarized in this notebook:
  * [List of Family Features and EDA](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Feature_Families_EDA.ipynb).
  
The family of features explored from the full data set prior cleaning are as follows: 
- **Arriving/Departing Flight** - features describing the flight characteristics such as time of arrival and departure dates and times, traffic conditions, delay and cancellation statuses
- **Airport of Origin/Destination** - features describing airport characteristics such as the ID, name, type, and location of origin and destination of each flight records
- **Airplane/Airline** - features identifying airline and airplane
- **Weather** - features describing physical characteristics of weather conditions during the flight
- **Engineered Flight/Airport features** - features not in original data set but found to have stonger correlation with the target variable for better modeling. These are further discussed in sections 3.4 and 3.5.

<img src ='https://github.com/noriiii/W261_Final_Project_NB/blob/main/FeaturesFamily.png?raw=true'>


### 2.4 Feature Engineering

In addition to the base features in the provided data sets we created several features based on our research and intuition about other factors related to potential delays. The details of each engineered features, including the resultant distribution of each engineered feature, is included in the [feature engineering](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Feature%20engineering.ipynb) notebook. 
* *Delays in Last 4 hours*: Partitioned by each origin airport, count the number of flights that have been delayed >=15 minutes in the past 4 hours as that airport. 
* *Inbound Departure Delay*: Was the last flight with the same tail-number as the plane schdeduled for this departure delayed? 
* *Departure Volume*: How many standard deviations away from the average # of scheduled departures for each airport is the count of schedules depatures for today? 
* *Page-Rank*: How important is the airport to overall flight connectivity? 
* *Holidays*: How many days away is the next major US holiday from the scheduled flight date?

<img src="https://github.com/stackoverthrow/261section7group2/blob/main/feature_eng.jpeg?raw=true" width=70%>

The delays in last 4-hours and inbound departure delay features were achieved with a SQL windowing function as shown in the code block below. The flight data is first partitioned by origin airport, sorted by pre-departure time and then the given calculation (in this case sum of delays) is performed over a window defined by the 'BETWEEN' keyword as shown below. This general framework is very useful for time series feature engineering and was employed several times in the feature engineering process. 

<img src="https://github.com/stackoverthrow/261section7group2/blob/main/window_func.jpeg?raw=true" width=70%>

### 2.5 Feature Correlation

<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/correlation%20.png?raw=true" width=80%>

The raw weather parameters did not have a very strong correlation with departure delay as shown in the correlation matrix and sub-set barplot for departure delay time shown below. The most strongly correlated features were two of the engineered features we created in Phase 3, which are 'inbound_dep_delay_time' and 'delays_last_4_hrs'. This result is strong motivation to continue being more creative with additional feature engineering and manipulation into phase 4 to further enhance the predictive power beyond the available features, so we added two new engineered features, "page_rank" and "is_holiday."

**Top 5 Features:**

<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/topfeaturesnew.png?raw=true" width=40%>

### 2.6 Handling Imbalanced Dataset

In the case of the classification models, the 5-to-1 imbalance of the on-time vs delayed flight counts will result in challenges training model unsuccessfully. For phase 4, we chose to handle imbalanced dataset by majority down-sampling, minority oversampling and SMOTE oversampling and compared which of this three balancing method yields the best prediction modeling results. As summarized in the below table, the on-time vs delayed flight counts was successfuly balanced to a 1-to-1 ratio.

<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/ProportionTarget.png?raw=true">

**Majority Down-sampling**

The majority class of non-delayed flights was downsampled to more closely match the minority delayed flight count as shown in the chart below. 

<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/Undersampling.PNG?raw=true">

**Minority Over-sampling**

The minority class of delayed flights was oversampled to more closely match the majority delayed flight count as shown in the chart below.

<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/Oversampling.PNG?raw=true">

##3. Data Leakage

### 3.1 Definition of Leakage & Potential examples

**Data leakage** occurs when information outside training data set is used unintentionally to train a model, or information from the validation data set is used in the training process. This concern is particularly important in the case of time-series data where future information or data not available at the time of the prediction should not be used as a part of the model building process. 

While we did take care to implement a series based approach to splitting our training and validation sets as described in section 4.2 below, there are few details of the feature engineering process that could be considered 'sinful'. The window based functions are inherently okay due to their focus on only the window of data preceding the given row. The approach we took to definining the business of the airport through the 'volume_sigma' and 'page_rank' features, however, were a little bit naughty. We used the full data set to establish the baseline distribution of average # of flights for each airport, and the baseline standard deviation. Similarly the 'page_rank' values for each airport were arrived through anaylsis of the entire data set. One can argue that these distribution based features are subject to the law of large numbers after a year or so of data given the lack of changes in the distribution from year to year, and that the leakge is unlikely to change the overall distribution estimate significantly. That being said, a better approach would have been to extend the data set into 2014 or before and use that to establish the 'historical' distribution rather than using the full set. Neither the page-rank nor the volume_sigma feature ended up being very important compared to the window based features and were not actually included in the final best performing model, thus absolving us of our ML sins. 

### 3.2 Train-Test Split

The nature of the flight delay prediction task entails a limitation about the information known at the time of each prediction. A standard randomized split of the data rows into train and test sets would result in 'leakage' of future information into the training set for past predictions. At the same time the flight delay probability varies significantly over the course of the seasons of the year so it is critical that we maintain at least a single continuous year of data in each training fold during the cross-validation process. To accomplish this task we will simply sort the training data by time-stamp and then use a rolling time series split like shown in the first table below, in which we train on a window that increases by 1 year with each fold, while the validation year also indexes by one.

##4. Modeling Pipelines

### 4.1 Modeling Pipeline Summary

A simplified, high-level overview of the pipeline our team constructed is shown in the figure below. The execution of each key step a code level is included [here](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Regression_engineered_features.ipynb). The following also provide quick-links to the separated notebooks for each analysis. We smoothly integrate the pipeline more for data preparation and feature generation through dbutils.notebook.run commands in phase 4. 
  * [Regression with raw features](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Regression_no_engineered_features.ipynb).
  * [Regression with engineered features](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Regression_engineered_features.ipynb).
  * [Classification with engineered features](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Classification_pipeline_new_features.ipynb)


<img src="https://github.com/stackoverthrow/261section7group2/blob/main/3_pipeline.jpeg?raw=true" width=90%>

**Data Load & Pre-treatment**
* Remove sparse columns 
* Drop low % null sparse rows 
* Impute missing rows 


### 4.2 Families of input features
<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/SelectedFeaturesFamily.png?raw=true">

There are nineteen (19) input features used in modeling and they belong to the following feature families:
- **Engineered Flight/Airport** (3 features)
- **Arriving/Departing Flight** (1 feature) 
- **Weather** (15 features)

None of the features from Airport of Origin/Destination and Airplane/Airline families were used in modeling, as none of them were found to have strong correlation with the target variable.

The list of family features and their EDA are summarized in this notebook:
  * [List of Family Features and EDA](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Feature_Families_EDA.ipynb).

The settings considered for conducting experiments in our modeling pipeline are working under undersampled, oversampled and SMOTE oversampled data, as well as including and excluding engineered features. All were successfully implemented except SMOTE oversampling due to the difficulties when implementing this method. Hyperparameter discussion were discussed in the section 6 below.

### 4.3 Loss Function

**Regression**

For the sake of readability we have separated the code for data loading, processing, feature engineering and regression model development in separatre notebooks:
* [Regression with raw features](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Regression_no_engineered_features.ipynb).
* [Regression with engineered features](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Regression_engineered_features.ipynb).


The intiial primary goal of this project was to predict the actual flight departure delay time in minutes represented by the DEP_DELAY_NEW variable with only data available greater than two hours from the scheduled departure time. In the case of regression the loss function for gradient descent during model training was the Mean Square Error or MSE defined as: 

$$MSE = \frac{\sum_{i=1}^{n}{(y_i  -  \hat{y})^2}}{N} $$

We may also be interested in the coefficient of variation but not for the purposes of model training. The coefficient of variation, or R2 is defined as:

$$R^2 = 1 - \frac{Unexplained Variation}{Total Variation} = 1 - \frac{\sum_{i=1}^{n}{(y_i  -  \hat{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}} $$

**Classification**

For the sake of readability we have deparated the code for classification model development [here](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Classification_pipeline_new_features.ipynb).

During phase 1 and 2 we discovered the task of accurately predicting the delay time in minutes may be much more challenging than expected and suspected that a simple binary classification of whether a flight will or will not be delayed may be more successful. When switching to a classification task we will adjust our loss function from MSE to the categorical cross-entropy which is defined by the following relationship: 

$$Cross Entropy = -\frac{1}{M}\sum_{i=1}^{M}{y_i\log{\hat{y_i}} + (1-y_i)log(1 - \hat{y_i})}$$

In addition to the cross-entropy loss metric which will be used for the actual model training process we will also evaluate the effectiveness of our models using the area under precision-recall curve as well as the base, precision, recall, and F1 scores to evaluate the predictive power of our classifcation models. 

$$Precision = \frac{TP}{TP + FP}$$

$$Recall = \frac{TP}{TP + FN} $$

$$F1_{score} = \frac{2(Recall)(Precision)}{Recall + Precision}$$

### 4.4 Experiment Summary

<img src ="https://github.com/noriiii/W261_Final_Project_NB/blob/main/ExperimentSummary.png?raw=true">

As shown in the Experiment Summary table, there are a total of eighteen (18) experiments conducted between various regression and classification models using either the full dataset, downsampled data or oversampled data. Additionally, eight (8) experiments conducted for Multilayer Perceptron Neural Network on different network architectures for undersampled data, which are discussed further in the next section. The training and evaluation times for each experiement are also reported in the table. 

The experiments include regression models such as Linear, Decision Tree, Random Forest and Gradient Boosted Tree. All regression models were experimented with or without engineered features included. 

The models experimented for binary classification include Decision Tree, Random Forest, Gradient Boosted, Logistic Regression and Multilayer Perceptron Neural Network. All classification models were experimented under downsampling and oversampling conditions.

The experimental results are discussed in Section 7.

##5. Neural Network


As part of this project, we have experimented on different neural network architectures and also experimented with training at different epochs and using different optimizers. The image below shows the experimental set up and results.


* [Multiperceptron Experimental Databricks Notebook](https://github.com/soggiugiorgio/Portfolio_Projects/blob/main/Flight_Delays_Prediction/Classification_MLP_Expt.ipynb).


<img src ="https://github.com/noriiii/W261_Final_Project_NB/blob/main/NeuralNetwork.png?raw=true">

As can be seen above, the first three models shows that as we decrease the number of hidden layers from 3 to 1, the validation area under the ROC curve increases with a significant increase in value when we moved from three hidden layers to two hidden layers and of course out of these three models, the model with one hidden layer (19-38-Sigmoid-2-Softmax) gave the best results. This is because for this data, the model probably started to overfit the training data. Hence, only one hidden layer should be used with this data.

Since, we got a better result using one hidden layer (19-38-Sigmoid-2-Softmax) and using gradient descent as the optimizer, we decided to explore another optimizer called l-bfgs which is provided as part of the pyspark package to compare such output with the model created with gradient descent keeping all other factors constant.

l-bfgs is otherwise called limited-memory BFGS where BFGS is named after its creators: Broyden, Fletcher, Goldfarb, and Shanno. l-bfgs is a second order optimization algorithm (unlike gradient descent which is a first order optimization algorithm) found in the family of the quasi newton algorithms and simply used to help minimize the cost function.

Our result showed that keeping all other factors constant, using l-bfgs (19-38-Sigmoid-2-Softmax(l-bfgs, maxIter=100)) gave a better performance than using gradient descent (19-38-Sigmoid-2-Softmax) as the optimizer. Due to this result, we then decided to perform another set of experiments using one hidden layer, l-bfgs as the optimizer, keeping all other factors constant but changing the maxIter(epoch) value each time (50, 100, 150, 200, 300). Our result showed that an increase in the epochs (maxIter) translates to an increase in the model performance. However, as we move from epoch size of 250 to 300 (see image below), we started to see a slow upward trend which signifies that if we continue to increase the maxIter value, a point would likley set in where there would be no more increase in model performance. According to our experiments, **(19-38-Sigmoid-2-Softmax(l-bfgs, maxIter=300))** gave the best model performance on the test sets.

<img src ="https://github.com/YemiOlani/w261_images/blob/main/MLP_result_2.JPG?raw=true">

## 6. Results & Discussion

As shown on section 5.4, within the regression prediction task, the decision tree model with engineered features has the best overall performance in terms of relative balance of loss and training time. While the gradient-boosted tree model has the lowest loss, its loss is not significantly different from the decision tree model and it has a very significant training time. The random forest also performed slightly better but with a significant training time burden that does not appear to be worth the trouble. Hence, we conclude that our final prediction pipeline is the decision tree regression in phases 2, 3 and 4. The inclusion of the engineered features did slightly improve the loss compared to their exclusion.

However, none of the regression models performed better than 10% over the naive baseline. So on phase 4, we decided to focus on experimenting on binary classification model to find the best performance. In the classification task, the logistic regression performed the best in terms of AUC. While Random Forest and Multilayer Perceptron Neural Network (19-38-Sigmoid-2-Softmax(l-bfgs, maxIter=300)) models produced the highest AUC, the logistic regression has significantly lower amount of time to produce nearly the same AUC value that Random Forest, Gradient-Boosted and Multilayer Perceptron Neural Network models produce. Hence, our final classification pipeline for phase 4 is the logistic regression, which is a confirmation of what we also found in phase 3. The classification models typically performed better on oversampled data, but Logistic Regression trained and evaluated faster on undersampled data.

While we have sufficient amount of successful experiments to make our final conclusion, we had challenges encountered in all phases. For example, cancelling XgBoost due to programming error on Phase 2, Gradient-Boosted taking longer than 3 days for regression task on Phase 3, and having difficulty implementing SMOTE oversampling on Phase 4 due to programming errors and time-consuming runs. These challenges are motivations for us to continue to improve in our future work in experimenting and finding the best performing regression and classification models.

**Gap Analysis**

A brief summary of our analysis of our gaps from the leading groups is included in the figure below. 
 
<img src="https://github.com/noriiii/W261_Final_Project_NB/blob/main/GapAnalysis.png?raw=true" width=60%>

In comparison to other teams, our logistic regression produced the third highest AUC for both  cross fold validation and blind test set performances. Team “Section 5 Group 1” reported the highest AUC of >0.90, but is unclear which of their models yielded this result. Our logistic regression trained faster than most teams; however, it is not the fastest. “FP_Section4_Group4” trained their logistic regression model for 55 seconds only, which is six times faster than our model. Our Logistic Regression model is fastest in blind test evaluation time, tying with Section 5 Group 1. 

Hence, our logistic regression model is one of the best performers in terms of accuracy and speed, but there is room for improvement, which we can do in future work. This will include continuation of experimentations under SMOTE oversampling or other strategies. We will consider more hyperparameter tuning and more sophisticated models. The threats that we foresee are the significant amount of time to train and debug, as well as communication issues with other teams  for more accurate comparison.

## 7. Conclusions and Next Steps

U.S flights departure delays result in financial losses and customer dissatisfaction. Hence, the major focus of this project is to accurately report to the airline leadership team, a machine learning based prediction using airline and weather features of flight delays so as to plan alternatives for customers thereby increasing customer satisfaction. Our hypothesis is experimentally true about engineered features improving prediction modeling performance, so we will utilize this strategy. We considered decision tree for prediction task due to its short training and inference times while performing among the best. We considered logistic regression for classification task due to its short training and inference times while performing among the best. Our models are competitive, performing among the best when compared to other project teams. The result of this research would help airlines to give more accurate flight predictions under hectic schedule resulting in better customer experience.

For future work, we will expand feature engineering to consider the arrival airport, create graph features like the page-rank of the origin/desintation airport, expand the scope of hyper-parameter tuning and consider further down-sampling for gradient boosted decision tree training. We will also include more details around each model iteration such as the feature importance, and plots of precision/recall curves.

## 8. Citations

1. General. BTS. (n.d.). Retrieved November 30, 2022, from https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp